In [1]:
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from objetos_otimizacao_rede_neural import *
from lightning.pytorch.loggers import TensorBoardLogger
import os

In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 0
NUM_EPOCAS = 2

In [3]:
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs/")
treinador = L.Trainer(logger=False,enable_checkpointing=True,max_epochs=NUM_EPOCAS)

num_dados_de_entrada = 5
num_dados_de_saida = 1
neuronios_c1 = 10
neuronios_c2 = 20
neuronios_c3 = 10


minha_mlp = MLP(
    num_dados_de_entrada, neuronios_c1, neuronios_c2,neuronios_c3, num_dados_de_saida
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
k = []
for i in range(int(1/TAMANHO_TESTE)):
    print(i)
    dm = DataModule(TAMANHO_TESTE, SEMENTE_ALEATORIA,i)

    treinador.fit(minha_mlp, dm)

    minha_mlp.eval()
    dm.setup("test")

    with torch.no_grad():
        X_true = dm.X_teste

        y_true = dm.y_teste
        y_true = dm.y_scaler.inverse_transform(y_true)

        y_pred = minha_mlp(X_true)
        y_pred = dm.y_scaler.inverse_transform(y_pred)

        RMSE = root_mean_squared_error(y_true, y_pred)
    
    k.append(RMSE)

rmse_medio = (sum(np.array(k)**2)/int(1/TAMANHO_TESTE))**.5




0


C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\jose23038\OneDrive - ILUM ESCOLA DE CIÊNCIA\Semestre 3 - 2024.1\Redes-neurais-e-Algoritmos-Geneticos\Rede Neurais\Tarefas\Redes-Neurais\Otimização\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 501   
---------------------------------------
501       Trainable params
0         Non-trainable params
501       Total params
0.002     Total estimated

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\jose23038\OneDrive - ILUM ESCOLA DE CIÊNCIA\Semestre 3 - 2024.1\Redes-neurais-e-Algoritmos-Geneticos\Rede Neurais\Tarefas\Redes-Neurais\Otimização\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 501   
---------------------------------------
501       Trainable params
0         Non-trainable params
5

1
2
3
4
5
6


`Trainer.fit` stopped: `max_epochs=2` reached.
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\jose23038\OneDrive - ILUM ESCOLA DE CIÊNCIA\Semestre 3 - 2024.1\Redes-neurais-e-Algoritmos-Geneticos\Rede Neurais\Tarefas\Redes-Neurais\Otimização\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 501   
---------------------------------------
501       Trainable params
0         Non-trainable params
5

7
8
9


In [7]:
k

[319.46855145295734,
 280.3089118396176,
 264.8041085111298,
 275.2615991382594,
 263.7571449852689,
 352.36382378707486,
 299.2886560701901,
 264.84565250948435,
 268.0054748592615,
 327.2044158200473]

In [8]:
rmse_medio

293.0628063582051